In [48]:
# import required libraries
import pandas as pd
import yfinance as yf
import numpy as np
import math 

In [80]:
# get stock prices
ticker = 'TQQQ'
start_date = '2009-01-01'
end_date = '2019-01-01'

df = yf.download(ticker, start_date, end_date, threads= False)
df = df.reset_index()
df.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-02-11,0.813750,0.869792,0.811146,0.865104,0.862942,1728000
1,2010-02-12,0.841563,0.876146,0.836667,0.868646,0.866475,4300800
2,2010-02-16,0.889063,0.904375,0.875104,0.902292,0.900037,4809600
3,2010-02-17,0.914375,0.917813,0.900625,0.917604,0.915311,9590400
4,2010-02-18,0.916667,0.940521,0.911146,0.935000,0.932663,19430400


In [81]:
# parameter setup (default values in the original indicator)
length = 20
mult = 2
length_KC = 20
mult_KC = 1.5

In [82]:
# calculate Bollinger Bands
# moving average
m_avg = df['Close'].rolling(window=length).mean()
# standard deviation
m_std = df['Close'].rolling(window=length).std(ddof=0)
# upper Bollinger Bands
df['upper_BB'] = m_avg + mult * m_std
# lower Bollinger Bands 
df['lower_BB'] = m_avg - mult * m_std

In [83]:
# calculate Keltner Channel
# first we need to calculate True Range
df['tr0'] = abs(df["High"] - df["Low"])
df['tr1'] = abs(df["High"] - df["Close"].shift())
df['tr2'] = abs(df["Low"] - df["Close"].shift())
df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
# moving average of the TR
range_ma = df['tr'].rolling(window=length_KC).mean()
# upper Keltner Channel
df['upper_KC'] = m_avg + range_ma * mult_KC
# lower Keltner Channel
df['lower_KC'] = m_avg - range_ma * mult_KC

In [84]:
# check for 'squeeze'
df['squeeze_on'] = (df['lower_BB'] > df['lower_KC']) & (df['upper_BB'] < df['upper_KC'])
df['squeeze_off'] = (df['lower_BB'] < df['lower_KC']) & (df['upper_BB'] > df['upper_KC'])

In [85]:
# calculate momentum value***
highest = df['High'].rolling(window = length_KC).max()
lowest = df['Low'].rolling(window = length_KC).min()
m1 = (highest + lowest) / 2
df['value'] = (df['Close'] - (m1 + m_avg)/2)
fit_y = np.array(range(0,length_KC))
df['value'] = df['value'].rolling(window = length_KC).apply(lambda x : np.polyfit(fit_y, x, 1)[0] * (length_KC-1) + np.polyfit(fit_y, x, 1)[1], raw=True)

In [86]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,upper_BB,lower_BB,tr0,tr1,tr2,tr,upper_KC,lower_KC,squeeze_on,squeeze_off,value
0,2010-02-11,0.813750,0.869792,0.811146,0.865104,0.862942,1728000,NaN,NaN,0.058646,NaN,NaN,0.058646,NaN,NaN,False,False,NaN
1,2010-02-12,0.841563,0.876146,0.836667,0.868646,0.866475,4300800,NaN,NaN,0.039479,0.011042,0.028437,0.039479,NaN,NaN,False,False,NaN
2,2010-02-16,0.889063,0.904375,0.875104,0.902292,0.900037,4809600,NaN,NaN,0.029271,0.035729,0.006458,0.035729,NaN,NaN,False,False,NaN
3,2010-02-17,0.914375,0.917813,0.900625,0.917604,0.915311,9590400,NaN,NaN,0.017188,0.015521,0.001667,0.017188,NaN,NaN,False,False,NaN
4,2010-02-18,0.916667,0.940521,0.911146,0.935000,0.932663,19430400,NaN,NaN,0.029375,0.022917,0.006458,0.029375,NaN,NaN,False,False,NaN


In [87]:
# entry point for long position:
# 1. black cross becomes gray (the squeeze is released)
df['long_cond1'] = (df['squeeze_off'].shift(2) == False) & (df['squeeze_off'].shift() == True) 
# 2. bar value is positive => the bar is light green
df['long_cond2'] = df['value'].shift() > 0
df['enter_long'] = df['long_cond1'] & df['long_cond2']
# entry point for short position:
# 1. black cross becomes gray (the squeeze is released)
df['short_cond1'] = (df['squeeze_off'].shift(2) == False) & (df['squeeze_off'].shift() == True) 
# 2. bar value is negative => the bar is light red 
df['short_cond2'] = df['value'].shift() < 0
df['enter_short'] = df['short_cond1'] & df['short_cond2']

In [109]:
df = df.dropna()
df[df['enter_long'] == True].head()

,Date,Open,High,Low,Close,Adj Close,Volume,upper_BB,lower_BB,tr0,...,lower_KC,squeeze_on,squeeze_off,value,long_cond1,long_cond2,enter_long,short_cond1,short_cond2,enter_short
88,2010-06-18,1.033229,1.051458,1.020417,1.030729,1.028153,34896000,1.053382,0.816055,0.031041,...,0.836313,False,True,0.065687,True,True,True,True,False,False
250,2011-02-08,1.813229,1.851250,1.804167,1.851250,1.846624,33494400,1.840022,1.624551,0.047083,...,1.639646,False,True,0.049172,True,True,True,True,False,False
303,2011-04-26,1.881250,1.916667,1.869792,1.898125,1.893381,11198400,1.875693,1.631765,0.046875,...,1.662604,False,True,0.061289,True,True,True,True,False,False
594,2012-06-20,2.117917,2.137917,2.062917,2.113750,2.108468,48607200,2.107512,1.751155,0.075000,...,1.790333,False,True,0.155594,True,True,True,True,False,False
626,2012-08-06,2.242917,2.286250,2.228333,2.256250,2.250612,31903200,2.245348,1.873068,0.057917,...,1.905115,False,True,0.114450,True,True,True,True,False,False


In [89]:
take_profit = 0.3
stop_loss = 0.1
capital = 10000
position_size = 0.3

In [111]:
trades = []

for i in range(len(df)):
    
    if df['enter_long'].iloc[i] == True:
        trade = []
        print(i)
        for j in range(i+1, len(df)):
            price_of_stock = df['Open'].iloc[i+1]
            max_trade_amount = capital * position_size
            if price_of_stock > max_trade_amount:
                break
            max_num_of_stocks = max_trade_amount // price_of_stock
            print(max_num_of_stocks)
            order_size = max_num_of_stocks * price_of_stock
            capital = capital - order_size
            
            total_value = max_num_of_stocks * df['Open'].iloc[j]
            unrealized_pnl = total_value - order_size
            unrealized_pnl_p = unrealized_pnl / order_size
            
            if(unrealized_pnl_p >= take_profit or unrealized_pnl_p <= -stop_loss):
                if(unrealized_pnl >= take_profit):
                    trade.append(df['Date'].iloc[i+1])
                    trade.append(unrealized_pnl)
                    trade.append(unrealized_pnl_p)
                    trade.append(df["Date"].iloc[j+1])
                    trade.append(capital)
                    trades.append(trade)
                    
                if(unrealized_pnl_p <= -stop_loss):
                    trade.append(df['Date'].iloc[i+1])
                    trade.append(unrealized_pnl)
                    trade.append(unrealized_pnl_p)
                    trade.append(df["Date"].iloc[j+1])
                    trade.append(capital)
                    trades.append(trade)
                    
                capital = capital + unrealized_pnl + order_size
                break

50
212
265
556
588
617
759
889
917
1026
1030
1040
1220
1379
1546
1627
1701
1759
1775
2025
2027
2035
2139


In [105]:
for x in range(len(trades)):
        print(trades[x])

In [93]:
PnL = []
EnterDate = []
ExitDate = []
Trades = []
start_long = 1000
start_short = 0
long_fund = start_long
short_fund = start_short
current_ath = 0
capital = long_fund + short_fund

for i in range(len(df)):
    if df['buy_tqqq'].iloc[i] == True:
        trade = []
        j = 1
        for j in range(i + 1, len(df)):
            if df['close_tqqq'].iloc[j] == True:
                gain_abs = df['Open'].iloc[j + 1] - df['Open'].iloc[i+1]
                gain_per = round(gain_abs / df['Open'].iloc[i+1] * 100,1)
                long_fund = round(long_fund * (100 + gain_per)/100,2)
#                 PnL.append(gain_abs)
#                 EnterDate.append(df['Date'].iloc[i+1])
#                 ExitDate.append(df['Date'].iloc[j+1])
                trade.append("Long")
                trade.append(gain_per)
                trade.append(long_fund)
                trade.append(df['Date'].iloc[i+1])
                trade.append(df['Date'].iloc[j+1])
                Trades.append(trade)
                
                # enter shorts immediately with the long funds
#                 trade = []
#                 for k in range(j + 1, len(df)):
#                     if df['close_sqqq'].iloc[k] == True:
#                         gain_abs =  df['Open'].iloc[j+1] - df['Open'].iloc[k + 1] #need to add SQQQ
#                         gain_per = round(gain_abs / df['Open'].iloc[i+1] * 100,1)
#                         long_fund = round(long_fund * (100 + gain_per)/100,2)
#                         trade.append("Short*")
#                         trade.append(gain_per)
#                         trade.append(long_fund)
#                         trade.append(df['Date'].iloc[j+1])
#                         trade.append(df['Date'].iloc[k+1])
#                         Trades.append(trade)
#                         break

                break
                
    if df['buy_sqqq'].iloc[i] == True and df['ath'].iloc[i] > current_ath:
        trade = []
        if(df['Open'].iloc[i+2] > df['Open'].iloc[i+1]):
            gain_abs =  df['Open'].iloc[i+1] - df['Open'].iloc[i + 2] #need to add SQQQ
            gain_per = round(gain_abs / df['Open'].iloc[i+1] * 100,1)
            short_fund = round(short_fund * (100 + gain_per)/100,2)
            trade.append("Short")
            trade.append(gain_per)
            trade.append(short_fund)
            trade.append(df['Date'].iloc[i+1])
            trade.append(df['Date'].iloc[i+1])
            Trades.append(trade)
        else:
            for j in range(i + 1, len(df)):
                    if df['close_sqqq'].iloc[j] == True:
                        gain_abs =  df['Open'].iloc[i+1] - df['Open'].iloc[j + 1] #need to add SQQQ
                        gain_per = round(gain_abs / df['Open'].iloc[i+1] * 100,1)
                        short_fund = round(short_fund * (100 + gain_per)/100,2)
                        trade.append("Short")
                        trade.append(gain_per)
                        trade.append(short_fund)
                        trade.append(df['Date'].iloc[i+1])
                        trade.append(df['Date'].iloc[j+1])
                        Trades.append(trade)
                        break
    
    current_ath = df['ath'].iloc[i]

KeyError: 'buy_tqqq'

In [746]:
for x in range(len(Trades)):
        print(Trades[x])

['Long', -4.1, 959.0, Timestamp('2011-08-02 00:00:00'), Timestamp('2011-08-03 00:00:00')]
['Long', -10.4, 859.26, Timestamp('2011-08-15 00:00:00'), Timestamp('2011-08-19 00:00:00')]
['Long', -5.7, 810.28, Timestamp('2011-08-25 00:00:00'), Timestamp('2011-09-06 00:00:00')]
['Long', -0.9, 802.99, Timestamp('2011-09-08 00:00:00'), Timestamp('2011-09-09 00:00:00')]
['Long', 32.7, 1065.57, Timestamp('2011-09-13 00:00:00'), Timestamp('2011-12-09 00:00:00')]
['Long', -8.2, 978.19, Timestamp('2011-12-28 00:00:00'), Timestamp('2012-01-17 00:00:00')]
['Long', 24.2, 1214.91, Timestamp('2012-01-18 00:00:00'), Timestamp('2012-04-10 00:00:00')]
['Long', -8.1, 1116.5, Timestamp('2012-04-26 00:00:00'), Timestamp('2012-05-09 00:00:00')]
['Long', -6.9, 1039.46, Timestamp('2012-05-11 00:00:00'), Timestamp('2012-05-15 00:00:00')]
['Long', -8.3, 953.18, Timestamp('2012-05-23 00:00:00'), Timestamp('2012-06-04 00:00:00')]
['Long', -3.2, 922.68, Timestamp('2012-06-07 00:00:00'), Timestamp('2012-06-08 00:00:00

In [747]:
print("ticker: ", ticker)
print("start date: ", Trades[0][3])
print("end date: ", Trades[len(Trades)-1][3])
print("total starting capital: ", capital)
print("long fund: ", start_long)
print("short fund: ", start_short)
total_returns = round(long_fund + short_fund,2)
print("final capital: ", total_returns)
print("final long: ", long_fund)
print("final short: ", short_fund)
pnl = round(long_fund + short_fund - capital,2)
print("pnl: ", pnl)
print("pnl%: ", round((pnl/capital * 100),2))

years = (Trades[len(Trades)-1][3] - Trades[0][3]).days/365
annualized = round((pow((total_returns / capital),1/years)-1)*100,2)
print("annualized%: ", annualized)

ticker:  TSLA
start date:  2011-08-02 00:00:00
end date:  2018-12-04 00:00:00
total starting capital:  1000
long fund:  1000
short fund:  0
final capital:  8700.78
final long:  8700.78
final short:  0.0
pnl:  7700.78
pnl%:  770.08
annualized%:  34.25


In [748]:
win_counts = 0
lose_counts = 0

for x in range(len(Trades)):
    if(Trades[x][1] > 0):
        win_counts += 1
    else:
        lose_counts += 1
        
print("win_counts: ", win_counts, "lose_counts: ", lose_counts)

win_counts:  31 lose_counts:  52


In [651]:
short_win_counts = 0
short_lose_counts = 0

for x in range(len(Trades)):
    if (Trades[x][0] == 'Short'):
        if(Trades[x][1] > 0):
            short_win_counts += 1
        else:
            short_lose_counts += 1
        
print("short_win_counts: ", short_win_counts, "short_lose_counts: ", short_lose_counts)

short_win_counts:  2 short_lose_counts:  3


In [ ]:
if (no_tqqq_pos & long_tqqq):
    enter_tqqq

if (tqqq_pos & (exit1_tqqq | exit2_tqqq)):
    close_tqqq
    
if (no_sqqq_pos & long_sqqq):
    enter_sqqq
    
if (sqqq_pos & (exit1_sqqq | exit2_sqqq)):
    close_sqqq